In [266]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [100]:
url='https://sports.news.naver.com/kbaseball/schedule/index?date=20230726&month={}&year=2023&teamCode='
response = requests.get(url)
soup = bs(response.text, 'lxml')

In [410]:
result_lst=[]
for month in range(4,10):
    url=f'https://sports.news.naver.com/kbaseball/schedule/index?date=20230728&month=0{month}&year=2023&teamCode='
    response = requests.get(url)
    soup = bs(response.text, 'lxml')
    div=soup.select('#calendarWrap>div')
    for i in div:
        date=i.select('td')[0].text.strip()
        for j in i.select('tr'):
            if j==i.select('tr')[0]:
                time=j.select('td')[1].text.strip()
                game=j.select('td')[2].text.strip().replace('\n','')
                if len(j.select('td')) == 3:
                    break
                elif j.select('td.add_state'): 
                    stadium=j.select('td')[4].text.strip()
                    bc=j.select('td')[3].text.strip()
                else:
                    stadium=j.select('td')[5].text.strip()
                    bc=j.select('td')[4].text.strip()
            else:
                time=j.select('td')[0].text.strip()
                game=j.select('td')[1].text.strip().replace('\n','')
                if len(j.select('td')) == 3:
                    break
                elif j.select('td.add_state'): 
                    stadium=j.select('td')[3].text.strip()
                    bc=j.select('td')[2].text.strip()
                else:
                    stadium=j.select('td')[4].text.strip()
                    bc=j.select('td')[3].text.strip()
           
            result_lst.append([date, time, game, stadium, bc])

result_lst


[['4.1 (토)', '14:00', '한화2:3키움', '고척', 'KBS2'],
 ['4.1 (토)', '14:00', 'KIA1:4SSG', '문학', 'MBC'],
 ['4.1 (토)', '14:00', 'LG6:11KT', '수원', 'MBC SPORTS+, SPOTVSPOTV2'],
 ['4.1 (토)', '14:00', '롯데10:12두산', '잠실', 'SBS'],
 ['4.1 (토)', '14:00', 'NC8:0삼성', '대구', 'KBS N SPORTS, SBS SPORTS'],
 ['4.2 (일)', '14:00', '한화6:7키움', '고척', 'SPOTV2'],
 ['4.2 (일)', '14:00', 'KIA9:5SSG', '문학', 'SBS SPORTS'],
 ['4.2 (일)', '14:00', 'LG10:9KT', '수원', 'SPOTV'],
 ['4.2 (일)', '14:00', '롯데2:0두산', '잠실', 'KBS N SPORTS'],
 ['4.2 (일)', '14:00', 'NC6:8삼성', '대구', 'MBC SPORTS+'],
 ['4.4 (화)', '18:30', '한화6:7삼성', '대구', 'MBC SPORTS+'],
 ['4.4 (화)', '18:30', 'KIAVSKT경기취소', '수원', '해당 경기는 현지 사정으로 취소되었습니다.'],
 ['4.4 (화)', '18:30', 'LG7:1키움', '고척', 'SPOTV2'],
 ['4.4 (화)', '18:30', '롯데1:3SSG', '문학', '네이버, wavveDAUM카카오, TVING스포키'],
 ['4.4 (화)', '18:30', 'NC0:1두산', '잠실', 'SPOTV'],
 ['4.5 (수)', '18:30', '한화VS삼성경기취소', '대구', '해당 경기는 현지 사정으로 취소되었습니다.'],
 ['4.5 (수)', '18:30', 'KIAVSKT경기취소', '수원', '해당 경기는 현지 사정으로 취소되었습니다.'],
 ['4.5 (수)',

In [405]:
for lst in result_lst:
    print(lst[2])


한화2:3키움
KIA1:4SSG
LG6:11KT
롯데10:12두산
NC8:0삼성
한화6:7키움
KIA9:5SSG
LG10:9KT
롯데2:0두산
NC6:8삼성
한화6:7삼성
KIAVSKT경기취소
LG7:1키움
롯데1:3SSG
NC0:1두산
한화VS삼성경기취소
KIAVSKT경기취소
LG1:2키움
롯데VSSSG경기취소
NC9:3두산
한화8:1삼성
KIAVSKT경기취소
LG5:0키움
롯데VSSSG경기취소
NC2:6두산
KT7:1롯데
두산4:1KIA
SSG7:3한화
삼성2:7LG
키움0:2NC
KT7:3롯데
두산6:7KIA
SSG7:5한화
삼성1:2LG
키움5:11NC
KT3:5롯데
두산3:2KIA
SSG3:0한화
삼성2:3LG
키움1:6NC
한화5:4KIA
KT0:1NC
LG5:6롯데
SSG5:4삼성
키움4:6두산
한화0:2KIA
KT1:5NC
LG12:8롯데
SSG3:0삼성
키움VS두산경기취소
한화5:1KIA
KT10:3NC
LG7:8롯데
SSG9:11삼성
키움9:2두산
한화7:7KT
KIA2:9키움
롯데7:9삼성
NC1:2SSG
두산4:13LG
한화7:2KT
KIA1:6키움
롯데9:5삼성
NC1:0SSG
두산1:3LG
한화2:14KT
KIA0:2키움
롯데1:9삼성
NC10:5SSG
두산10:5LG
KIA5:7롯데
NC6:4LG
두산2:0한화
SSG2:4KT
삼성6:4키움
KIA6:0롯데
NC5:7LG
두산6:7한화
SSG2:5KT
삼성9:5키움
KIA3:5롯데
NC4:9LG
두산5:1한화
SSG8:5KT
삼성1:6키움
KT6:10두산
LG4:2한화
롯데3:2NC
삼성4:5KIA
키움1:3SSG
KT1:5두산
LG3:0한화
롯데10:6NC
삼성2:6KIA
키움2:3SSG
KT1:1두산
LG6:7한화
롯데5:3NC
삼성3:5KIA
키움7:9SSG
한화VS롯데경기취소
KT0:1키움
NC6:0KIA
두산VS삼성경기취소
SSG4:5LG
한화1:8롯데
KT2:13키움
NC0:6KIA
두산0:1삼성
SSG5:3LG
한화0:3롯데
KT1:3키움
NC0:5KIA
두산6:7삼성
S

In [315]:
#calendarWrap > div:nth-child(1) > table > tbody > tr:nth-child(1) > td:nth-child(3)
//*[@id="calendarWrap"]/div[1]/table/tbody/tr[1]/td[3]

In [335]:
j.select('span')

[<span class="td_date"><strong>6.5</strong> (월)</span>,
 <span class="td_hour">-</span>,
 <span class="td_none">프로야구 경기가 없습니다.</span>]

In [332]:
print(date_lst)

NameError: name 'date_lst' is not defined

In [275]:
pd.set_option('display.max_rows', None)
df_res = pd.DataFrame(columns = ['날짜', '시간', '경기', '중계/기록', '중계방송사', '구장'])
for month in range(4, 10) :
    url = f"https://sports.news.naver.com/kbaseball/schedule/index?date=20230727&month=0{month}&year=2023&teamCode="
    soup = requests.get(url)
    for table in pd.read_html(soup.text) :
        if len(table.columns) == 7:
            df_res = pd.concat([df_res, pd.DataFrame(table.iloc[:, :-1].values, columns = df_res.columns)])

df_res

,날짜,시간,경기,중계/기록,중계방송사,구장
0,4.1 (토),14:00,한화 2:3 키움,NaN,KBS2,고척
1,4.1 (토),14:00,KIA 1:4 SSG,NaN,MBC,문학
2,4.1 (토),14:00,LG 6:11 KT,NaN,"MBC SPORTS+, SPOTVSPOTV2",수원
3,4.1 (토),14:00,롯데 10:12 두산,NaN,SBS,잠실
4,4.1 (토),14:00,NC 8:0 삼성,NaN,"KBS N SPORTS, SBS SPORTS",대구
0,4.2 (일),14:00,한화 6:7 키움,NaN,SPOTV2,고척
1,4.2 (일),14:00,KIA 9:5 SSG,NaN,SBS SPORTS,문학
2,4.2 (일),14:00,LG 10:9 KT,NaN,SPOTV,수원
3,4.2 (일),14:00,롯데 2:0 두산,NaN,KBS N SPORTS,잠실
4,4.2 (일),14:00,NC 6:8 삼성,NaN,MBC SPORTS+,대구
